In [ ]:
from data_download_tool import download_data, summarize_s3_structure
from annotation_viewer import plot_annotations, plot_annotation_subset
from pandas_statistics import file_path_loader
from plots import *
from aws_s3_viewer import S3Client
import os

def get_folders_in_directory(directory_path):
  if not os.path.exists(directory_path):
    return []
  folders = [
      entry.name for entry in os.scandir(directory_path) if entry.is_dir()
  ]
  return folders


# Clear Preprocessed Images

In [ ]:
storage_path = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data"


for file in get_folders_in_directory(storage_path):
    #Local file handling tool
    print(f"Num Samples: {len(local_files)}")
    local_files = file_path_loader(file)
    local_files.clear_cache()


# Recalculate All Statistics

In [ ]:
storage_path = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data"

for file in get_folders_in_directory(storage_path):
    #Local file handling tool
    print(f"Num Samples: {len(local_files)}")
    local_files = file_path_loader(file)
    local_files.recalculate_statistics()
